In [2]:
import pandas as pd
import numpy as np

from datetime import timedelta

## Training Set Generation for the IHC Webservice

     After granting you access to the Webservice, you will have the option to upload a training set of customer journeys (CSV or JSON file) that we will use to tailer the IHC model to your business case. You also need to specify a “Conv_Type_ID” for this training set, thereby you can tailor the IHC model to different types of conversions in your business context (e.g. new and returning customers).
      This notebook will be covering up how a simple dataset with site traffic information can be transformed to a suitable customer journey dataset for IHC Webservice in three steps. In the first step we are going to split the data in different types of conversions. The second step will describe how filtering and cleaning the conversions and sessions for the training set generation can be done. In the final step, we will transform the dataset into train ready customer journey set for IHC Webservice.


In [4]:
## loading datasets and parsing date columns
df_site_traffic_1 = pd.read_csv("datasets/site_traffic_1.csv", parse_dates = ["Event_Datetime", "Conv_Datetime"])
df_site_traffic_2 = pd.read_csv("datasets/site_traffic_2.csv", parse_dates = ["Event_Datetime", "Conv_Datetime"])
## merging dataset
df_site_traffic = pd.concat([df_site_traffic_1, df_site_traffic_2]).reset_index()

df_site_traffic

,index,Conv_ID,Session_ID,Market,Order_Num,Event_Datetime,Channel,Count_Pages,Count_Detail_Pages,Count_Checkout_Pages,Signed_in,Conv_Datetime
0,0,a583370491,fd48fae34a99d88f3594,NL,3,2020-09-16 10:25:14,SEA - Non Brand,14,3,5,0,2021-04-05 09:47:59
1,1,a583370491,c75735dfb653fbcb0751,NL,3,2020-09-16 10:47:57,Direct,2,1,1,0,2021-04-05 09:47:59
2,2,a583370491,93a8cc70c5174c5c20dc,NL,3,2020-09-17 21:31:12,Direct,3,1,0,0,2021-04-05 09:47:59
3,3,a583370491,3d84bf6d39285ee284c7,NL,3,2020-09-17 21:48:09,SEA - Brand,11,0,1,0,2021-04-05 09:47:59
4,4,a583370491,7abdcda5c8a3c0d664a0,NL,3,2020-09-17 22:03:02,Direct,1,1,1,0,2021-04-05 09:47:59
...,...,...,...,...,...,...,...,...,...,...,...,...
1671646,830274,7c2e38003e,b95d1970fd0d51c833c5,DE,0,2022-05-09 22:44:23,Shopping - Non Brand,1,1,0,0,2022-05-09 23:35:00
1671647,830275,7c2e38003e,7b881551365e9dc2d35c,DE,0,2022-05-09 22:47:22,Shopping - Non Brand,3,2,0,0,2022-05-09 23:35:00
1671648,830276,7c2e38003e,7cfb982d77ab1dd1aa77,DE,0,2022-05-09 23:17:51,Shopping - Non Brand,5,2,0,0,2022-05-09 23:35:00
1671649,830277,7c2e38003e,d70674adb5ff2fcd25bf,DE,0,2022-05-09 23:33:24,Shopping - Non Brand,7,2,0,0,2022-05-09 23:35:00


## Step 1: Conversion Types

     As we can see here in the pivot table below, we have two different markets, namely, NL and DE. Also, we can group the customers by their status, whether they have made a conversion before or not. So, we can split this dataset into four conversion types.

In [5]:
df_site_traffic.assign(Order_Status = np.where(df_site_traffic.Order_Num ==0, "new", "return"))\
               .pivot_table(columns=["Order_Status"], index=["Market"], aggfunc="nunique", values = "Conv_ID")

Order_Status,new,return
Market,,
DE,23069,22786
NL,23009,22879


In [4]:
## grouping the dataset with groupby function
grouped = df_site_traffic.assign(Order_Status = np.where(df_site_traffic.Order_Num ==0, "new", "return"))\
                         .groupby(["Market", "Order_Status"])
grouped.groups.keys()

dict_keys([('DE', 'new'), ('DE', 'return'), ('NL', 'new'), ('NL', 'return')])

     Our site traffic data contains four different conversion types, after that I want to continue with one of them, new customers from NL Market. However, with the provided data you can try out different conversion types by selecting a different group from the grouped dataframe above. 

In [5]:
## now for the simplicity lets choose one conversion type and continue with it
df_nl_new = grouped.get_group(("NL", 'new'))

## Step 2: Cleaning the Data

### Sessions After Upper Limit - Conversion

     Usually, our data can contain sessions, that took place after the conversion. That means that the customer has started a whole new journey, and we have to eliminate those sessions. We can say that conversion date is the upper limit for our dataset to filter out sessions.

In [6]:
mask_after_conversion = (df_nl_new['Event_Datetime'] <= df_nl_new['Conv_Datetime']) 
print(f"We have {(~mask_after_conversion).sum()} sessions after conversion events")


We have 104958 sessions after conversion events


### Sessions Before the Lower Limit

     Some sessions happened too long ago relative to the conversion date so they are not worth to be considered. Here we are creating another mask, which helps us to identify sessions, that took place more than 120 days ago from the conversion. Thus, we can say that the "conversion date minus 120 days" is the lower limit for our dataset to filter out sessions.

In [7]:
mask_long_ago = (df_nl_new['Event_Datetime'] >= (df_nl_new['Conv_Datetime'] - timedelta(days=120)))
print(f"{(~mask_long_ago).sum()} sessions happened 120 days ago from the conversion events")


15914 sessions happened 120 days ago from the conversion events


    Here we combine the Upper and the Lower limits to get the correct data.

In [8]:
mask_combined = (mask_after_conversion&mask_long_ago)
print(f"{(~mask_combined).sum()} sessions were filtered out")

df_nl_new = df_nl_new[mask_combined]

120872 sessions were filtered out


### Dropping Duplicates and NaN values

     It is also very important to delete duplicated rows with the same conversion id and session id. Before continuing with the training, we have to also eliminate all the NaN values.

In [9]:
## NaN values
df_nl_new.isna().sum()

Conv_ID                 0
Session_ID              0
Market                  0
Order_Num               0
Event_Datetime          0
Channel                 0
Count_Pages             0
Count_Detail_Pages      0
Count_Checkout_Pages    0
Signed_in               0
Conv_Datetime           0
Order_Status            0
dtype: int64

In [10]:
## droping NaNs
df_nl_new =df_nl_new.dropna()

In [11]:
## duplicates
mask_duplicated = df_nl_new.duplicated(subset = ["Conv_ID", "Session_ID"])
print(f"{(mask_duplicated).sum()} rows duplicated")

df_nl_new = df_nl_new[~mask_duplicated]

0 rows duplicated


     We applied these maskings to whole dataset. Now it is time to look at some statistics, and then we will continue with creating our training and test sets.

### Channel and Session Statistics

     In this table we can see occurance frequencies of channels, as well as influenced conversion counts for each channel. For example, our dataset has 16291 sessions with Email channel and 5435 customer journeys contain at least one Email channel inside.

In [12]:
df_nl_new.groupby("Channel").agg(
    Sessions = ("Session_ID", "count"),
    Influenced_Conversions = ("Conv_ID", "nunique")
)

,Sessions,Influenced_Conversions
Channel,,
Affiliate,3083,1389
Direct,37126,12423
Display,3,1
Email,16291,5435
Google Remarketing,1,1
Influencers,32,20
Referral,6865,4826
SEA - Brand,15068,8061
SEA - Non Brand,8529,5397


     Now let's look at the session statistics of customer journeys. Around %7 of customer journeys has only 1 session. In addition, conversions with only 2 sessions are %12 of our data. I will exclude those sessions from my training and test set, because they are two simple and do not carry to much information.
     Also %99 percent (0.99 quantile) of customer journeys has less than 44 sessions, which is 99th percentile our dataset. I will consider the other 1 percent as outliers and remove them from training data as well.

In [13]:
def grouping(sessions, a):
    if sessions==1:
        return '1'
    elif sessions==2:
        return '2'
    elif sessions==3:
        return '3'
    elif (sessions >3) &(sessions<a):
        return f'3< session <{a}'
    else:
        return f">{a}"
    
df_ses = df_nl_new.Conv_ID.value_counts().rename("session_count").reset_index()

## we will use this in order to extract super long customer journeys from our training and test set
quantile = df_ses.session_count.quantile(0.99)
df_ses["Group"] = df_ses.session_count.apply(grouping, a= int(quantile) )

df_ses_stats = df_ses.pivot_table(index = "Group", aggfunc = ['mean', "count"], values = "session_count")\
                     .droplevel(axis =1, level = 1)\
                     .rename(columns= {"mean":"Mean Number of Sessions", "count":"Conversion Count"}, index= {"session_group": "Session Count"})

df_ses_stats["Fraction of Conversion"] = df_ses_stats["Conversion Count"]/df_ses_stats["Conversion Count"].sum() 
df_ses_stats.round(2)


,Mean Number of Sessions,Conversion Count,Fraction of Conversion
Group,,,
1,1.00,1574,0.07
2,2.00,2854,0.12
3,3.00,3152,0.14
3< session <44,8.60,15190,0.66
>44,65.88,239,0.01


### Selection of Conversion IDs for Train and Test Splitting

     When it comes to training data, it is always better approach to train at least 15k customer journeys. The more homogenous your dataset is, the more consistent your results will be and the less training data you need to get those. For this example, I will choose 3000 random conversions for the evaluation. The rest is going to be my training set.

In [14]:
df_ses.head()

,index,session_count,Group
0,e5df2abddf,160,>44
1,5c320b3446,148,>44
2,6fa40e5a5a,143,>44
3,9d44f657cf,141,>44
4,c424698632,135,>44


     As I mentioned above, we can extract the conversions, that happened within 1 or 2 sessions from our training set because they are not significant for our parameter training and evaluating. It is also good for parameter training to ignore super long customer journeys. But we will keep them while evaluating the test set.

In [15]:
## masking the journeys with at least 3 sessions.
df_ses_gr_two = df_ses[df_ses.session_count >2]

In [16]:
## test set first
test_set_ids = df_ses_gr_two['index'].sample(n = 3000, random_state = 191)
print(f"Test set will contain {test_set_ids.size} conversions")

Test set will contain 3000 conversions


In [17]:
##train set
#masking
mask_session = (df_ses_gr_two.session_count<quantile) & (~df_ses_gr_two["index"].isin(test_set_ids))
train_set_ids = df_ses_gr_two[mask_session]["index"]

print(f"Train set will contain {train_set_ids.size} conversions")

Train set will contain 15381 conversions


## Step 3: Dataset Transformation

### Conversion Status Columns

     I would like to add one last column to my dataset. We extract all of the sessions after conversions, so we are sure that very last session in the each customer journey is the conversion session. To show where the conversion session is, I will add a boolean column to my data. This will help the function below to identify which session is the conversion session.

In [18]:
df_nl_new['Conv_Status'] = df_nl_new.groupby('Conv_ID').Event_Datetime\
                                    .apply(lambda x: x == x.max()).astype(int)

In [19]:
df_nl_new.head()

,Conv_ID,Session_ID,Market,Order_Num,Event_Datetime,Channel,Count_Pages,Count_Detail_Pages,Count_Checkout_Pages,Signed_in,Conv_Datetime,Order_Status,Conv_Status
113,4f61b4581e,001276b07b535e0b95b1,NL,0,2020-12-21 23:01:02,Direct,1,1,0,0,2021-04-14 22:47:22,new,0
114,4f61b4581e,e14c441da334edb3f666,NL,0,2020-12-26 17:41:41,SEA - Non Brand,1,1,0,0,2021-04-14 22:47:22,new,0
115,4f61b4581e,3ea5678e8945a0ad8fbf,NL,0,2020-12-29 18:54:14,Email,3,2,1,0,2021-04-14 22:47:22,new,0
116,4f61b4581e,1eb9dd9b26c2877dea8f,NL,0,2020-12-29 23:17:01,SEO,12,7,0,0,2021-04-14 22:47:22,new,0
117,4f61b4581e,bd9da5bb8c714ab202a2,NL,0,2021-01-01 12:03:46,Direct,3,2,1,0,2021-04-14 22:47:22,new,0


    When transforming the dataset, we will only keep the features which requiered by IHC Webservice. They are:
      
     - conversion id: Unique ID for related conversion; 
     
     - session id: Unique session ID; 
     
     - timestamp: Time of the session; 
     
     - channel_label: Channel used in the session; 
     
     - holder_engagement: Identifier if the session is eligible for holder phase; 
     
     - closer_engagement: Identifier if the session is eligible for closer phase;
     
     - conversion: Identifier whether conversion happened in this session or not. 

### Session Level Classification for Holder and Close Phase

     For sure, you best know your products and your platform steps in the conversion funnel. That's why, you need to quantify if the customer’s session engagement is eligible for the Holder or Closer phase. Here are some example rules;
         
       - Holder Engagement = 1, if there are at least two detail page views or at least one checkout related page view made, or interacted with the site signed in;
          
       - Closer Engagement =1, if there is at least one checkout related page view or conversion happened this session.

In [20]:
def create_cj(df):
    """
    -should be used as an input function to pandas DataFrame.apply(func, axis = 1).
    -transforms each row in dataframe to json records

    """
    session = {}
    
    ## session informations
    session["conversion_id"] = df.Conv_ID
    session["session_id"] = df.Session_ID
    session["timestamp"] = df.Event_Datetime
    session["channel_label"] = df.Channel
    
    #holder engagement
    session["holder_engagement"] = 1 if (df.Count_Detail_Pages>=2
                                         or df.Count_Checkout_Pages>=1 
                                         or df.Signed_in>=1) else 0
   
    #closer engagement
    session["closer_engagement"] = 1 if (df.Count_Checkout_Pages>=1 
                                         or df.Conv_Status) else 0
    
    #conversion
    session["conversion"] = 1 if df.Conv_Status else 0
    return session

In [21]:
records = df_nl_new.apply(create_cj, axis=1).values
df_nl_new_transformed = pd.DataFrame.from_records(records)

### Make Sure IDs are String

     The ID columns (conversion ID and session ID) in our dataset must be string values. To be sure, it is always better to convert them to string. In the end, this is what our dataset should look like

In [22]:
df_nl_new_transformed.conversion_id = df_nl_new_transformed.conversion_id.astype(str)
df_nl_new_transformed.session_id = df_nl_new_transformed.session_id.astype(str)

In [23]:
df_nl_new_transformed.head()

,conversion_id,session_id,timestamp,channel_label,holder_engagement,closer_engagement,conversion
0,4f61b4581e,001276b07b535e0b95b1,2020-12-21 23:01:02,Direct,0,0,0
1,4f61b4581e,e14c441da334edb3f666,2020-12-26 17:41:41,SEA - Non Brand,0,0,0
2,4f61b4581e,3ea5678e8945a0ad8fbf,2020-12-29 18:54:14,Email,1,1,0
3,4f61b4581e,1eb9dd9b26c2877dea8f,2020-12-29 23:17:01,SEO,1,0,0
4,4f61b4581e,bd9da5bb8c714ab202a2,2021-01-01 12:03:46,Direct,1,1,0


##  Parameter Training

     Finally, our customer journey sets (train and test) for the parameters "NL market and new customers" are ready. You can now create a new conversion type for this parameter in the IHC Webservice and train the dataset. 
     After that, you can compute the test set on this parameter. If you want to learn how to make API calls to the IHC Webservice, please check the "IHC 3 - Api Request" notebook. 


In [24]:
##you can save the dataframes as csv for further use 
df_nl_new_train = df_nl_new_transformed[df_nl_new_transformed.conversion_id.isin(train_set_ids)]
df_nl_new_test = df_nl_new_transformed[df_nl_new_transformed.conversion_id.isin(test_set_ids)]

In [25]:
df_nl_new_train.to_csv("datasets/cj_nl_new_train.csv", index  =False)
df_nl_new_test.to_csv("datasets/cj_nl_new_test.csv", index  =False)

In [212]:
df_nl_new_test

,conversion_id,session_id,timestamp,channel_label,holder_engagement,closer_engagement,conversion
62,b5455601a5,6f248ba6a7b27d85a599,2021-04-04 23:34:51,Shopping - Non Brand,0,0,0
63,b5455601a5,c37d81c07c8656907024,2021-04-04 23:36:18,SEO,1,0,0
64,b5455601a5,aa3ddaf73f8c6064bce9,2021-04-04 23:39:17,Shopping - Non Brand,1,1,0
65,b5455601a5,d98f8e5b45ee011db2d7,2021-04-05 09:32:30,Direct,1,1,1
86,12c7fb69ef,ee500aae2f2c0347c2b6,2020-12-26 12:54:39,Social Paid,0,0,0
...,...,...,...,...,...,...,...
163098,c87dbc0746,d754cd778bb3b448eba7,2022-05-09 11:01:33,Referral,1,0,0
163099,c87dbc0746,7d470a0bf0ee40a24c8a,2022-05-09 11:08:35,Referral,1,1,0
163100,c87dbc0746,e962fd64fb20c5916de3,2022-05-09 11:11:04,Referral,1,1,0
163101,c87dbc0746,9c03dd44bbe9bf8e2f45,2022-05-09 11:15:59,Direct,1,1,0


In [213]:
df_nl_new_train

,conversion_id,session_id,timestamp,channel_label,holder_engagement,closer_engagement,conversion
53,5e7c48c4d1,8ddbf8111b61918fd496,2020-12-08 10:02:28,Shopping - Non Brand,0,0,0
54,5e7c48c4d1,3b1303ff57f098b79098,2020-12-08 10:05:10,SEO,0,0,0
55,5e7c48c4d1,3add0d4ae6446f6c86c9,2020-12-09 17:25:58,Direct,1,1,0
56,5e7c48c4d1,c3173fe46e9ce1fe394a,2020-12-16 13:43:48,SEA - Non Brand,0,0,0
57,5e7c48c4d1,66ca090c795bee65a215,2021-01-05 14:28:32,Direct,1,0,0
...,...,...,...,...,...,...,...
163142,e0aa5b5cb8,5ab9c230d4b50d0982d2,2022-05-09 22:57:28,Shopping - Non Brand,0,0,0
163143,e0aa5b5cb8,ff685dd81bc78712f3d2,2022-05-09 23:02:14,SEO,0,0,0
163144,e0aa5b5cb8,7c4890824229f06f1df7,2022-05-09 23:03:08,Shopping - Non Brand,1,0,0
163145,e0aa5b5cb8,1615ca74eda13d5dac29,2022-05-09 23:08:38,Shopping - Non Brand,1,0,0
